In [103]:
# !pip install datasets
# !pip install Pillow
# !pip install numpy
# !pip install matplotlib
# !pip install import_ipynb
# !pip install tensorflow
# !pip install tensorflow.keras

# # !pip install torch
# # !pip install torchvision
# # !pip install scikit-image
# # !pip install scipy
# !pip install easyocr

# #library from etl
# %pip install datasets
# %pip install numpy
# %pip install Pillow
# %pip install opencv-python
# %pip install json

In [104]:
import easyocr
import matplotlib.pyplot as plt
from datasets import load_dataset
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
import json

In [105]:
from google.colab import userdata

# Retrieve the GitHub token from userdata
myGithub = userdata.get('githubKey')

!git clone https://{myGithub}@github.com/fitristachan/invoice-image-processing.git /content/invoice-image-processing
!git -C invoice-image-processing pull

fatal: destination path '/content/invoice-image-processing' already exists and is not an empty directory.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
Unpacking objects: 100% (3/3), 432 bytes | 108.00 KiB/s, done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
From https://github.com/fitristachan/invoice-image-processing
   4ebd865..171a1a0  main       -> origin/main
Updating 4ebd865..171a1a0
Fast-forward
 data_etl_invoice.ipynb | 26 +++++++++++++-------------
 1 file changed, 13 insertions(+), 13 deletions(-)


In [106]:
import sys
import os

sys.path.append('/content/invoice-image-processing')
print(os.listdir('/content/invoice-image-processing'))

!jupyter nbconvert --to script '/content/invoice-image-processing/data_etl_invoice.ipynb' --output data_etl

['.git', 'invoice-image-processing.code-workspace', 'requirements.txt', 'Dataset', 'data_etl_invoice.ipynb', 'preprocessing.ipynb', 'data_etl.py', '__pycache__', 'model_ver_1.ipynb']
[NbConvertApp] Converting notebook /content/invoice-image-processing/data_etl_invoice.ipynb to script
[NbConvertApp] Writing 7835 bytes to /content/invoice-image-processing/data_etl.py


**Text Extraction**

> using easyocr

> support bahasa indonesia and english





In [107]:
reader = easyocr.Reader(['en', 'id'])

def is_receipt(text, logic='AND'):
    required_keywords = ['total', 'amount', 'jumlah', 'price', 'harga']
    text_lower = text.lower()

    #diubah nanti jadinya ada yang and ada yang or
    if logic == 'AND':
        return all(keyword in text_lower for keyword in required_keywords)
    elif logic == 'OR':
        return any(keyword in text_lower for keyword in required_keywords)
    else:
        raise ValueError("Invalid logic parameter. Use 'AND' or 'OR'.")


def process_image(image, logic='AND'):
    # Convert PIL Image to numpy array
    image_np = np.array(image)

    # Extract text using EasyOCR
    results = reader.readtext(image_np)

    # Combine all detected text into a single string
    extracted_text = ' '.join([result[1] for result in results])

    # Check if the extracted text is a receipt
    if is_receipt(extracted_text, logic=logic):
        print("This is a valid receipt.")
        print("Extracted Text:")
        print(extracted_text)
    else:
        print("This is not a valid receipt.")

Training

In [108]:
from data_etl import DatasetReceipt

In [110]:
# Inisialisasi dataset
dataset = DatasetReceipt()

# Ambil sampel data
sample = dataset[0]
print("Sample Output:")
print("Image Shape:", sample["image"].shape)  # Akan menampilkan (600, 600)
print("Label:", sample["label"])

# Konversi gambar ke TensorFlow tensor
image = tf.convert_to_tensor(sample["image"], dtype=tf.float32)
image = tf.expand_dims(image, axis=-1)  # Tambahkan channel untuk CNN
print("Image Tensor Shape:", image.shape)  # Akan menampilkan (600, 600, 1)

NameError: name 'tf' is not defined

In [ ]:
# Lanjutkan dengan pembuatan model TensorFlow/Keras
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(600, 600, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # Sesuaikan dengan jumlah kelas
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("Model Summary:")
model.summary()

In [ ]:
# model.fit(padded, labels, epochs=10, batch_size=32)